In [1]:
from astropy.io import fits
from astropy.table import Table
from math import log10
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy.misc import imresize
from scipy import misc
from scipy.ndimage import zoom
from scipy.ndimage.interpolation import rotate
import glob

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten

from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import rmsprop

import random
import pdb

from sklearn.utils import shuffle

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

from keras.models import model_from_json
from keras.models import model_from_yaml
from keras.optimizers import SGD

/Users/marchuertascompany/anaconda3/envs/astrophd_tutorial/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Function to read images

In [10]:
def extract_thumb(im,x,y,size):
    if size %2==0:
        size = size+1
    up_x=int(x-size/2)
    dow_x=int(x+size/2)
    up_y=int(y-size/2)
    dow_y=int(y+size/2)
    res=im[up_x:dow_x,up_y:dow_y]        
    return res 

# to read data
def read_data(pathin,pathsave,maxim):
    size_im=69
    size_crop=207
  

    data=fits.getdata(pathin+'Nair_Abraham_cat.fit',1)
    idcat=data['dr7objid']
    ttype=data['TType']
    
    #define the morphologies we want to retrieve
    m=ttype*0-1
    #m[np.where((ttype>=0) & ((ttype<=3)))]=1
    #m[np.where(((ttype<0) & (ttype>=-5)) | ((ttype>3) & (ttype<=10)))]=0
    m[np.where((ttype>=-5) & (ttype<=0))]=0
    m[np.where((ttype>0) & (ttype<=10))]=1
      
    D=np.zeros([maxim,size_im,size_im,3])  #input tensor - images dimensions + color channels
    Y=np.zeros(maxim) #label vector
    idvec=np.zeros([maxim], dtype=np.long)
    
    iteri=-1;
    numim=0;
    numim_init=numim
    nplace=0  #location 1st galaxy to be read 
    catalog=Table(data)

    while iteri<maxim-1:
        try:
            numgal=idcat[numim]
            namegal=str(numgal)+"_GZOO_.jpg"        
            scidata = misc.imread(pathin+'/cutouts_jpeg_all/'+namegal)
            f=numim
            
            print('reading: '+namegal)
            
        except:
            print("Galaxy number %d is missing" % (numim))
            print(namegal)
            numim += 1
            continue
        
        lx,ly, lz=scidata.shape
        #wrong shape - ignore image
        if lx < 256 or ly<256 or m[f]<0:
            numim += 1
            continue

        if lx<size_im:
            numim += 1
            continue

        
        scidata = extract_thumb(scidata,int(lx/2.0),int(ly/2.0),size_crop) # take only a cutout of 207*207 pixels
        scidata=zoom(scidata, [1/3.,1./3,1], order=3)  #keep 1/3 pixels to speed up
        
        
        iteri=iteri+1
        
        #scidata = np.transpose(scidata) #tranpose image
        
        D[iteri,:,:,:]=scidata  #add image to the input tensor

        Y[iteri]=m[f] #update the label
        
        idvec[iteri]=idcat[f] 

        if iteri%100 ==0:
            print("Saving example")
            misc.imsave(pathsave+"examples_stamps/"+namegal,scidata)
            
         
        numim=numim+1

        
        Y = Y.squeeze()

    # this is to avoid reading all images at every training
    print("Saving image and target vector")
    np.save(pathsave+"image_vector_Sab_"+str(maxim)+".npy",D) 
    np.save(pathsave+"target_vector_Sab_"+str(maxim)+".npy",Y)
    np.save(pathsave+"ID_vector_Sab_"+str(maxim)+".npy",idvec) 

    return D,Y

# Function for model definition

In [7]:
#only two convolutional layers - this can be changed

def CNN_Nair(img_channels, img_rows, img_cols):
    dropoutpar=0.5
    depth=16 #32   
    nb_dense = 64
    # SGD parameters [when using SGD optimizer]
    #lr=0.001   #0.001
    #decay=0
    #momentum=0.9   #0.9
    #nesterov=True

    model = Sequential()
    model.add(Convolution2D(depth, 3, 3,init='orthogonal',activation='relu',border_mode='same', input_shape=( img_rows, img_cols,img_channels)))
    model.add(Dropout(dropoutpar))
    model.add(Convolution2D(depth*2, 5, 5,init='orthogonal',activation='relu',border_mode='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(nb_dense, activation='relu'))
    model.add(Dropout(dropoutpar)) 
    model.add(Dense(1, init='uniform', activation='softmax'))
    print("Compilation...")
    #sgd = SGD(lr=lr, decay=decay, momentum=momentum, nesterov=True) #uncomment to use sgd
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    print("... done!")
    print("Model Summary")
    print("===================")
    model.summary()
    return model

# Function to train

In [8]:
def train_convnet_Nair(X,Y,ntrain,nval,test_name):


        
    # train params - hardocded for simplicity
    batch_size = 30 #64
    nb_epoch = 50
    data_augmentation = True
    
    
    ind=random.sample(range(0, ntrain+nval-1), ntrain+nval-1)
    X_train = X[ind[0:ntrain],:,:,:]   
    X_val = X[ind[ntrain:ntrain+nval],:,:,:]
    Y_train = Y[ind[0:ntrain]]
    Y_val = Y[ind[ntrain:ntrain+nval]]

   
    # input image dimensions
    img_rows, img_cols = X_train.shape[1:3]
    img_channels = 3

    print(img_rows,img_cols)
    pdb.set_trace()
    
    ### Right shape for X
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols,img_channels)
    X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols,img_channels)


    #Avoid more iterations once convergence
    patience_par=10
    earlystopping = EarlyStopping( monitor='val_loss',patience = patience_par,verbose=0,mode='auto' )
    modelcheckpoint = ModelCheckpoint(test_name+"_best.hd5",monitor='val_loss',verbose=0,save_best_only=True)


    #build model
    model=CNN_Nair(img_channels, img_rows, img_cols)




    if not data_augmentation:
        print('Not using data augmentation.')
        history = model.fit(X_train, Y_train,
                            batch_size=batch_size,
                            nb_epoch=nb_epoch,
                            validation_data=(X_val, Y_val),
                            shuffle=True,
                            verbose=verbose, callbacks=[earlystopping, modelcheckpoint])
    else:
        print('Using real-time data augmentation.')

        # this will do preprocessing and realtime data augmentation
        datagen = ImageDataGenerator(
            featurewise_center=False, 
            samplewise_center=False, 
            featurewise_std_normalization=False, 
            samplewise_std_normalization=False,
            zca_whitening=False, 
            rotation_range=45,
            width_shift_range=0.05,  
            height_shift_range=0.05, 
            horizontal_flip=True,
            vertical_flip=True,
            zoom_range=[0.75,1.3])  

        
        datagen.fit(X_train)
        
        history = model.fit_generator(
                    datagen.flow(X_train, Y_train, batch_size=batch_size),
                    samples_per_epoch=X_train.shape[0],
                    nb_epoch=nb_epoch,
                    validation_data=(X_val, Y_val),
                    callbacks=[ earlystopping, modelcheckpoint]
                )



    print("Saving model...")
    # save weights
    model.save_weights(test_name,overwrite=True)
    
        
    
    return test_name

# Function to test

In [14]:
def test_convnet_Nair(X,model_name):
    
    # input image dimensions
    img_rows, img_cols = X.shape[1:3]
    img_channels = 3
    X = X.reshape(X.shape[0], img_rows, img_cols,img_channels)
    
    #====== load model & predict=======

    print("Loading weights", model_name)
    
    model=CNN_Nair(img_channels, img_rows, img_cols)
    model.load_weights(model_name)
    Y_pred = model.predict_proba(X)


    return Y_pred

# Read

In [11]:
READ_IMAGES=True
LOAD_NPY=False


maxim=50  #number of images to read in D, Y vectors

pathin="/Users/marchuertascompany/Documents/teaching/big_data_ED/morphology/"
pathsave = "/Users/marchuertascompany/Documents/teaching/big_data_ED/morphology/Sab/"
nparams=1

#file to save model weights
model_name=pathsave+"Nair_Sab.hd5"

## reading

if READ_IMAGES:
    print("Reading images")
    print("====================")
    D,Y=read_data(pathin,pathsave,maxim)  #read images
    
if LOAD_NPY:
    print("Loading D, Y")
    D=np.load(pathsave+"image_vector_TType_"+str(maxim)+".npy")
    Y=np.load(pathsave+"target_vector_TType_"+str(maxim)+".npy")
    ID=np.load(pathsave+"ID_vector_TType_"+str(maxim)+".npy")    

Reading images
reading: 587748927626149924_GZOO_.jpg
Saving example
reading: 587748927626870899_GZOO_.jpg


/Users/marchuertascompany/anaconda3/envs/astrophd_tutorial/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/Users/marchuertascompany/anaconda3/envs/astrophd_tutorial/lib/python3.6/site-packages/ipykernel_launcher.py:80: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


reading: 587748927627722875_GZOO_.jpg
reading: 587722981742084144_GZOO_.jpg
reading: 587722981744640177_GZOO_.jpg
reading: 587722981744771128_GZOO_.jpg
reading: 587722981745295552_GZOO_.jpg
reading: 587722981745426489_GZOO_.jpg
reading: 587722981747392587_GZOO_.jpg
reading: 587722981748048006_GZOO_.jpg
reading: 587722981750276247_GZOO_.jpg
reading: 587722981750931635_GZOO_.jpg
reading: 587722981754011901_GZOO_.jpg
reading: 587722981755977976_GZOO_.jpg
reading: 587748928160530448_GZOO_.jpg
reading: 587748928160858191_GZOO_.jpg
reading: 587748928162562264_GZOO_.jpg
reading: 587748928166428807_GZOO_.jpg
reading: 587722982278496424_GZOO_.jpg
reading: 587722982278758580_GZOO_.jpg
reading: 587722982278889623_GZOO_.jpg
reading: 587722982282035411_GZOO_.jpg
reading: 587722982283214886_GZOO_.jpg
reading: 587722982284460121_GZOO_.jpg
reading: 587722982284460272_GZOO_.jpg
reading: 587722982284525580_GZOO_.jpg
reading: 587722982285967469_GZOO_.jpg
reading: 587722982289834254_GZOO_.jpg
reading: 587

# Train

In [12]:
ntrain=int(D.shape[0]*8/10)
nval=int(ntrain/10)    
D, Y, = shuffle(D,Y,random_state=0)  #change order so that we do not use always the same objects to train/test


print("Training Model")
print("====================")
model_name=train_convnet_Nair(D,Y,ntrain,nval,model_name)

Training Model
69 69
> <ipython-input-8-bbbe2b32a78e>(26)train_convnet_Nair()
-> X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols,img_channels)
(Pdb) c


/Users/marchuertascompany/anaconda3/envs/astrophd_tutorial/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="relu", input_shape=(69, 69, 3..., padding="same", kernel_initializer="orthogonal")`
  
/Users/marchuertascompany/anaconda3/envs/astrophd_tutorial/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same", kernel_initializer="orthogonal")`
  app.launch_new_instance()
/Users/marchuertascompany/anaconda3/envs/astrophd_tutorial/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="softmax", kernel_initializer="uniform")`


Compilation...
... done!
Model Summary
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 69, 69, 16)        448       
_________________________________________________________________
dropout_1 (Dropout)          (None, 69, 69, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 69, 69, 32)        12832     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 34, 34, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 36992)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2367552   
_________________________________________________________________
dropout_2 (Dropout)          (None, 6

/Users/marchuertascompany/anaconda3/envs/astrophd_tutorial/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/Users/marchuertascompany/anaconda3/envs/astrophd_tutorial/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=(array([[[..., callbacks=[<keras.ca..., steps_per_epoch=1, epochs=50)`


Epoch 1/50
1/1 [==============================] - 2s 2s/step - loss: 6.9084 - acc: 0.5667 - val_loss: 5.3141 - val_acc: 0.6667
Epoch 2/50
1/1 [==============================] - 0s 451ms/step - loss: 7.9712 - acc: 0.5000 - val_loss: 5.3141 - val_acc: 0.6667
Epoch 3/50
1/1 [==============================] - 0s 456ms/step - loss: 11.1597 - acc: 0.3000 - val_loss: 5.3141 - val_acc: 0.6667
Epoch 4/50
1/1 [==============================] - 1s 1s/step - loss: 5.8455 - acc: 0.6333 - val_loss: 5.3141 - val_acc: 0.6667
Epoch 5/50
1/1 [==============================] - 1s 1s/step - loss: 6.9084 - acc: 0.5667 - val_loss: 5.3141 - val_acc: 0.6667
Epoch 6/50
1/1 [==============================] - 0s 498ms/step - loss: 7.9712 - acc: 0.5000 - val_loss: 5.3141 - val_acc: 0.6667
Epoch 7/50
1/1 [==============================] - 1s 1s/step - loss: 6.9084 - acc: 0.5667 - val_loss: 5.3141 - val_acc: 0.6667
Epoch 8/50
1/1 [==============================] - 1s 638ms/step - loss: 7.9712 - acc: 0.5000 - val_lo

# Test

In [15]:


npred=D.shape[0]-(ntrain+nval)  #test sample size; 
pred_index=ntrain+nval          #test sample start index ;


print("Validating model")
print("====================")
Y_pred=test_convnet_Nair(D[pred_index:pred_index+npred,:,:,:],model_name) 
   

Validating model
Loading weights /Users/marchuertascompany/Documents/teaching/big_data_ED/morphology/Sab/Nair_Sab.hd5
Compilation...
... done!
Model Summary
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 69, 69, 16)        448       
_________________________________________________________________
dropout_3 (Dropout)          (None, 69, 69, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 69, 69, 32)        12832     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 34, 34, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 36992)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)            

/Users/marchuertascompany/anaconda3/envs/astrophd_tutorial/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="relu", input_shape=(69, 69, 3..., padding="same", kernel_initializer="orthogonal")`
  
/Users/marchuertascompany/anaconda3/envs/astrophd_tutorial/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same", kernel_initializer="orthogonal")`
  app.launch_new_instance()
/Users/marchuertascompany/anaconda3/envs/astrophd_tutorial/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="softmax", kernel_initializer="uniform")`
